<a href="https://colab.research.google.com/github/dirkpmueller/100-days-of-code-exercises/blob/main/hackthebox.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##**Hack the box**
- set up VMware
- downloaded and installed Parrot OS Security
- downloaded and run OpenVPN


##**Tier 0 Lab1 - MEOW**


##**Tier 0 Lab2 - FAWN**

- FTP
- Network
- Protocols
- Reconnaissance
- Anonymous/Guest Access
